# Midterm

## Data

First we will begin by loading in the two datasets and merging them. Before merging the datasets it was important to check the format of the two datasets. Thankfully, they two datasets had the same columns and order of columns, making merging the datasets very simple.

In [207]:
#| echo: False
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_selector, ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [45]:
data1 = pd.read_csv("/Users/Bnkes/Desktop/GitHub/AdvancedMachineLearning/Data/MidtermData/data1.csv")
data1 = data1.drop("Unnamed: 0", axis = 1)
data2 = pd.read_csv("/Users/Bnkes/Desktop/GitHub/AdvancedMachineLearning/Data/MidtermData/data2.csv")
data2 = data2.drop("Unnamed: 0", axis = 1)
merged_data = pd.concat([data1, data2])
merged_data = merged_data.dropna()

In [47]:
merged_data.head()

,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,19556,Female,Loyal Customer,52,Business travel,Eco,160,5,4,3,...,5,5,5,5,2,5,5,50,44.0,satisfied
1,90035,Female,Loyal Customer,36,Business travel,Business,2863,1,1,3,...,4,4,4,4,3,4,5,0,0.0,satisfied
2,12360,Male,disloyal Customer,20,Business travel,Eco,192,2,0,2,...,2,4,1,3,2,2,2,0,0.0,neutral or dissatisfied
3,77959,Male,Loyal Customer,44,Business travel,Business,3377,0,0,0,...,1,1,1,1,3,1,4,0,6.0,satisfied
4,36875,Female,Loyal Customer,49,Business travel,Eco,1182,2,3,4,...,2,2,2,2,4,2,4,0,20.0,satisfied


Now that we have the data in one large dataframe, it is important to check for missing values, remove the rows with the missing values, and examine the dataset. We will do this by examining the numerical columns with describe() and the categorical columns with value_counts() to determine the makeup of the dataset. Here, we are looking to see if the dataset is unbalanced in any way.

In [50]:
merged_data.describe()

,id,Age,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes
count,129487.000000,129487.000000,129487.000000,129487.000000,129487.000000,129487.000000,129487.000000,129487.000000,129487.000000,129487.000000,129487.000000,129487.000000,129487.000000,129487.000000,129487.000000,129487.000000,129487.000000,129487.000000,129487.000000
mean,64958.335169,39.428761,1190.210662,2.728544,3.057349,2.756786,2.976909,3.204685,3.252720,3.441589,3.358067,3.383204,3.351078,3.631886,3.306239,3.642373,3.286222,14.643385,15.091129
std,37489.781165,15.117597,997.560954,1.329235,1.526787,1.401662,1.278506,1.329905,1.350651,1.319168,1.334149,1.287032,1.316132,1.180082,1.266146,1.176614,1.313624,37.932867,38.465650
min,1.000000,7.000000,31.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,32494.500000,27.000000,414.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,3.000000,3.000000,3.000000,2.000000,0.000000,0.000000
50%,64972.000000,40.000000,844.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,4.000000,4.000000,4.000000,4.000000,4.000000,3.000000,4.000000,3.000000,0.000000,0.000000
75%,97415.500000,51.000000,1744.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,5.000000,4.000000,4.000000,4.000000,5.000000,4.000000,5.000000,4.000000,12.000000,13.000000
max,129880.000000,85.000000,4983.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,1592.000000,1584.000000


In [52]:
merged_data["Gender"].value_counts()

Gender
Female    65703
Male      63784
Name: count, dtype: int64

In [54]:
merged_data["Customer Type"].value_counts()

Customer Type
Loyal Customer       105773
disloyal Customer     23714
Name: count, dtype: int64

In [56]:
merged_data["Type of Travel"].value_counts()

Type of Travel
Business travel    89445
Personal Travel    40042
Name: count, dtype: int64

In [58]:
merged_data["Class"].value_counts()

Class
Business    61990
Eco         58117
Eco Plus     9380
Name: count, dtype: int64

In [60]:
merged_data["satisfaction"].value_counts()

satisfaction
neutral or dissatisfied    73225
satisfied                  56262
Name: count, dtype: int64

After examining the data and removing the missing values, we see that the classes in the dataset are fairly balanced, except for the "Class" and "Customer Type" columns. However, both of these columns have a large enough amount of each observation for them to be valuable in their current state. Now we will create the train test split and label encode the "satisfaction" column (target variable).

In [63]:
X = merged_data.drop(["id", "satisfaction"], axis = 1)
y = merged_data["satisfaction"]
y = LabelEncoder().fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y)

## Modeling

### Random Forest

To begin our modeling efforts, we will start with a relatively simple model and use a random forest to set a baseline value.

In [65]:
ct = ColumnTransformer(
    [
        ("dummify", OneHotEncoder(sparse_output=False, handle_unknown="error", drop = "first"), make_column_selector(dtype_include=object)),
        ("standardize", StandardScaler(), make_column_selector(dtype_include=np.number))
    ]
)

In [75]:
my_pipeline = Pipeline(
    [
        ("preprocessing", ct),
        ("forest", RandomForestClassifier(n_jobs=-1))
    ]
)

parameters = {
    "forest__min_samples_leaf": [1, 5, 7, 10, 50, 100],
    "forest__min_samples_split": [2, 5, 7, 10, 50, 100],
    "forest__ccp_alpha": [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0],
    "forest__n_estimators": [10, 50, 100, 250]
}

gscv = GridSearchCV(my_pipeline, parameters, cv = 5, scoring='f1', n_jobs = 1, verbose = 1)
gscv_fitted = gscv.fit(X, y)
test_scores = gscv_fitted.cv_results_["mean_test_score"]
gscv_fitted.best_estimator_

Fitting 5 folds for each of 1008 candidates, totalling 5040 fits


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('dummify',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000023F5B857950>),
                                                 ('standardize',
                                                  StandardScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000023F5B856950>)])),
                ('forest',
                 RandomForestClassifier(ccp_alpha=1e-06, n_estimators=250,
                                        n_jobs=-1))])

In [105]:
pd.DataFrame(gscv_fitted.cv_results_).sort_values("mean_test_score", ascending=False).head(5)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_forest__ccp_alpha,param_forest__min_samples_leaf,param_forest__min_samples_split,param_forest__n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
3,1.517488,0.021486,0.095592,0.004397,0.000001,1,2,250,"{'forest__ccp_alpha': 1e-06, 'forest__min_samp...",0.957236,0.954815,0.956236,0.955986,0.958305,0.956516,0.001181,1
867,1.481664,0.014298,0.094117,0.004529,0,1,2,250,"{'forest__ccp_alpha': 0, 'forest__min_samples_...",0.956012,0.954233,0.956815,0.956490,0.958341,0.956378,0.001326,2
7,1.485192,0.009276,0.093221,0.001962,0.000001,1,5,250,"{'forest__ccp_alpha': 1e-06, 'forest__min_samp...",0.956471,0.953807,0.956710,0.955652,0.958708,0.956269,0.001589,3
147,1.618124,0.019660,0.095247,0.002784,0.00001,1,2,250,"{'forest__ccp_alpha': 1e-05, 'forest__min_samp...",0.956239,0.953841,0.956788,0.955340,0.959018,0.956245,0.001708,4
11,1.476815,0.093765,0.093062,0.001231,0.000001,1,7,250,"{'forest__ccp_alpha': 1e-06, 'forest__min_samp...",0.956043,0.954295,0.956236,0.956443,0.958057,0.956215,0.001197,5


In [90]:
my_pipeline = Pipeline(
    [
        ("preprocessing", ct),
        ("forest", RandomForestClassifier(n_jobs=-1, min_samples_leaf=1, min_samples_split=2, n_estimators=250, ccp_alpha = 1e-6))
    ]
)

fitted_pipeline = my_pipeline.fit(X_train, y_train)

In [107]:
y_pred = fitted_pipeline.predict(X)

cm = confusion_matrix(y_true = y, y_pred = y_pred)

cm_df = pd.DataFrame(cm, index=["Actual Satisfied", "Actual Neutral/Unsatisfied"], columns=["Predicted Satisfied", "Predicted Neutral/Unsatisfied"])

cm_df

,Predicted Satisfied,Predicted Neutral/Unsatisfied
Actual Satisfied,72860,365
Actual Neutral/Unsatisfied,855,55407


In [103]:
f1 = f1_score(y_true=y, y_pred = y_pred)
print(f"F1 Score for Random Forest Classifier: {f1}")

F1 Score for Random Forest Classifier: 0.9891104486138137


As can be seen above, the Random Forest model perfomed extremely well at predicting both classes. I believe that it will be difficult to beat this score, but I am still going to try by utilizing XGBoost. I believe that this model would be good enough to deploy in a real world situation, as it only misclassified .94% of the cases it met. Something interesting that we can do with a random forest is examine which features had the largest effect on the classifiation and we will do that below.

In [202]:
feature_values = fitted_pipeline.named_steps["forest"].feature_importances_
feature_names = fitted_pipeline.named_steps["preprocessing"].get_feature_names_out()
feature_importance = dict(zip(feature_names, feature_values))
sorted(feature_importance.items(), key=lambda x: x[1], reverse=True)

[('standardize__Online boarding', 0.17296286989085474),
 ('standardize__Inflight wifi service', 0.13891119480785502),
 ('dummify__Type of Travel_Personal Travel', 0.10021395934511979),
 ('dummify__Class_Eco', 0.08050940052541579),
 ('standardize__Inflight entertainment', 0.056386566816473624),
 ('standardize__Seat comfort', 0.04920807682200429),
 ('standardize__Leg room service', 0.0443323534762351),
 ('standardize__Flight Distance', 0.03893627153693184),
 ('standardize__Ease of Online booking', 0.03793611903271542),
 ('standardize__Age', 0.03466918561452034),
 ('dummify__Customer Type_disloyal Customer', 0.03463225974140702),
 ('standardize__On-board service', 0.0317939525112709),
 ('standardize__Cleanliness', 0.025385410375828987),
 ('standardize__Checkin service', 0.024966769947811045),
 ('standardize__Baggage handling', 0.023861613437767572),
 ('standardize__Inflight service', 0.023024020758961133),
 ('standardize__Departure/Arrival time convenient', 0.01736785797947192),
 ('standa

Examining the top 5 most important features according to the random forest, we can see that the most satisfied customers are those who can check into their flights online (online boarding), have access to inflight wifi, are traveling for personal reasons, are in economy class, and have access to inflight entertainment. Most of these are fairly self explanatory, people prefer to have autonomy over checking into their flights and having access to some form of entertainment during their flight. However, I was surprised to see that economy class passengers tended to be more satisfied than Economy Plus and Business Class passengers. I believe this is due to two major factors. The first is that people who are flying business class are most likely doing so for work and probably fly often. I believe this group of people would be less likely to have their satisfaction level affected by comfort and would be more likely to care about delays. The other factor is that people who fly business class and economy plus are more likely to be satisfied with the airline that they are flying on already, as most frequent fliers stick to a single airline where they can get rewards for flying often.

### XGBoost

Seeing as the Random Forest model above generated very good results, I do not expect there to be a large increase in performance by using XGBoost, but I want to try it and see if I can get an even better classifier. One unfortunate downside of XGBoost is that it is not an interpretable model.

In [217]:
my_pipeline = Pipeline(
    [
        ("preprocessing", ct),
        ("xgb", XGBClassifier(n_jobs=-1))
    ]
)

parameters = {
    "xgb__n_estimators": [10, 50, 100, 250],
    "xgb__reg_alpha": [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0],
    "xgb__reg_lambda": [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0]
}

gscv = GridSearchCV(my_pipeline, parameters, cv = 5, scoring='f1', n_jobs = 1, verbose = 1)
gscv_fitted = gscv.fit(X, y)
test_scores = gscv_fitted.cv_results_["mean_test_score"]
gscv_fitted.best_estimator_

Fitting 5 folds for each of 196 candidates, totalling 980 fits


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('dummify',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000023F187EA590>),
                                                 ('standardize',
                                                  StandardScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000023F28162ED0>)])),
                ('xgb',
                 XGBClassifie...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=100, n_jobs=-1,
                               num_parallel_tree=None, random_state=None, ...))])

In [219]:
pd.DataFrame(gscv_fitted.cv_results_).sort_values("mean_test_score", ascending=False).head(5)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_xgb__n_estimators,param_xgb__reg_alpha,param_xgb__reg_lambda,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
114,0.346278,0.001939,0.048011,0.001789,100,0.0001,0.0001,"{'xgb__n_estimators': 100, 'xgb__reg_alpha': 0...",0.957664,0.953774,0.957507,0.958217,0.958979,0.957228,0.001802,1
112,0.343855,0.002825,0.047611,0.000490,100,0.0001,0.000001,"{'xgb__n_estimators': 100, 'xgb__reg_alpha': 0...",0.957664,0.953774,0.957507,0.957750,0.958979,0.957135,0.001760,2
113,0.349479,0.005390,0.048011,0.001674,100,0.0001,0.00001,"{'xgb__n_estimators': 100, 'xgb__reg_alpha': 0...",0.957664,0.953774,0.957507,0.957750,0.958979,0.957135,0.001760,2
118,0.358383,0.008033,0.049211,0.001166,100,0.0001,0,"{'xgb__n_estimators': 100, 'xgb__reg_alpha': 0...",0.957664,0.953774,0.957507,0.957750,0.958979,0.957135,0.001760,2
129,0.347278,0.006179,0.048211,0.000400,100,0.01,0.001,"{'xgb__n_estimators': 100, 'xgb__reg_alpha': 0...",0.957594,0.955149,0.958009,0.956400,0.957988,0.957028,0.001107,5


In [221]:
my_pipeline = Pipeline(
    [
        ("preprocessing", ct),
        ("xgb", XGBClassifier(n_jobs=-1, n_estimators=100, reg_alpha = .0001, reg_lambda = .0001))
    ]
)

fitted_pipeline = my_pipeline.fit(X_train, y_train)

In [223]:
y_pred = fitted_pipeline.predict(X)

cm = confusion_matrix(y_true = y, y_pred = y_pred)

cm_df = pd.DataFrame(cm, index=["Actual Satisfied", "Actual Neutral/Unsatisfied"], columns=["Predicted Satisfied", "Predicted Neutral/Unsatisfied"])

cm_df

,Predicted Satisfied,Predicted Neutral/Unsatisfied
Actual Satisfied,72192,1033
Actual Neutral/Unsatisfied,2382,53880


In [225]:
f1 = f1_score(y_true=y, y_pred = y_pred)
print(f"F1 Score for Random Forest Classifier: {f1}")

F1 Score for Random Forest Classifier: 0.9692826624690802


Somewhat surprisingly, the XGBoost model did less well than the Random Forest Model. While the difference in performance is small, the Random Forest also has the added benefit of being an interpretable model. Between the two models, I would pick the Random Forest as the preferred model for predicting these classes.

## Final Thoughts

As this data was collected via a survey, it is likely a semi-biased data set. While there are no major issues with datasets generated by surveys, it is important to consider some of the implicit biases that surveys (especially optional ones) can have. For example, it is unlikely that very unsatisfied customers are going to take a voluntary survey, especially if it is provided to them while they are traveling. Another problem with this survey is that we do not know what airlines these customers are flying on which could introduce further bias. Another issue with this survey is when the data was collected. A passenger who had a severly negative experience may remember it, but those with just average experiences may rate their satisfaction higher than they would if prompted when the flight ended. If I were to administer this survey, I would do it at the end of a flight, ideally when taxing to the gate, as it would get the most unfiltered opinions of the passengers on the flight. It is difficult to generalize my results to anyone in particular, as we do not know what airlines these passengers were flying on and what other confounding variables could have affected their scores. If the survey data was taken in some extremely busy airports, people may experience a longer arrival/departure delay but care about it less than if they experienced the same delay at a smaller airport. If I knew which airline these survey results were from, it would be possible for the airline to use these models, especially the random forest, to improve aspects of their business to create a more satisfied customer base. Even without knowing which airline, all airlines could look at the random forest results to determine which features tend to have the largest impact on customer satisfaction.